## Webscrapping for Trump Rallies

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [8]:
URL = "https://www.rev.com/blog/transcript-tag/donald-trump-rally-transcripts"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

temp = soup.findAll('div', attrs = {'class' : 'fl-post-column'})
transcript_links1 = [re.search(('(?<=href=")(.*)(?="><div class)'),str(i.find('a'))).group(0) for i in temp]


URL = "https://www.rev.com/blog/transcript-tag/donald-trump-rally-transcripts/page/2"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

temp = soup.findAll('div', attrs = {'class' : 'fl-post-column'})
transcript_links2 = [re.search(('(?<=href=")(.*)(?="><div class)'),str(i.find('a'))).group(0) for i in temp]


transcript_links = transcript_links1 + transcript_links2

### Parsing Rallies

In [53]:
combined_df = pd.DataFrame(columns = ["name","text","time"])

for rallies in transcript_links:
    URL = rallies
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    temp = soup.find('div', attrs = {'class' : 'fl-callout-text'}).find_all('p')
    moderator = temp[0].text + " " + temp[1].text

    pars = []

    for i, p in enumerate(soup.find('div', attrs = {'class' : 'fl-callout-text'}).find_all('p')):
        if (p.text != '\xa0'):
            pars.append(p.text)
            
    rally = {
    "name": [],
    "time": [],
    "text": [],
    }

    for text in pars:
        rally["name"].append(text.split(': ',1)[0])
        rally["time"].append(text.split(': ',1)[1].split(' ',1)[0])
        rally["text"].append(text.split(': ',1)[1].split(' ',1)[1])
            
    rally_df = pd.DataFrame(rally)
    
    filename = "{}.csv".format(' '.join(URL.split('-')[2:4]))    
    rally_df.to_csv("../data/trump-rallies/"+filename, index = False)
    
    rally_name = filename[:-4]
    rally_df["rally"] = [rally_name for i in range(len(rally_df))]
    combined_df = combined_df.append(rally_df, ignore_index = True, sort = False)

## Combining Transcripts

In [54]:
combined_df.to_csv("../data/trump-rallies/combined.csv",index = False)
combined_df

,name,text,time,rally
0,Donald Trump,"I want to thank you very much. North Carolina,...",(02:45),charlotte north
1,Donald Trump,This is an incredible time for our nation. Our...,(03:39),charlotte north
2,Donald Trump,"By the way, look at this. Look back there, loo...",(04:19),charlotte north
3,Donald Trump,There’s never been a time where the Republican...,(04:59),charlotte north
4,Donald Trump,"Jobs are booming in our country, incomes are s...",(05:53),charlotte north
...,...,...,...,...
2587,Donald Trump,"We are one movement, one movement, think of th...",(01:32:04),maga event
2588,Donald Trump,"We are so respected, you have no idea how our ...",(01:32:55),maga event
2589,Donald Trump,"You’ve all heard it many times, right, we will...",(01:33:54),maga event
2590,Donald Trump,"Look at her beautiful hat, it looks very good ...",(01:34:46),maga event


In [55]:
combined_df["text"]

0       I want to thank you very much. North Carolina,...
1       This is an incredible time for our nation. Our...
2       By the way, look at this. Look back there, loo...
3       There’s never been a time where the Republican...
4       Jobs are booming in our country, incomes are s...
                              ...                        
2587    We are one movement, one movement, think of th...
2588    We are so respected, you have no idea how our ...
2589    You’ve all heard it many times, right, we will...
2590    Look at her beautiful hat, it looks very good ...
2591                                           (singing).
Name: text, Length: 2592, dtype: object